In [1]:
import requests
import pandas as pd
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px


# Step 1: Download the Dataset and Skeleton App using Python

In [2]:
dataset_url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"
app_url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/spacex_dash_app.py"


In [3]:
import pandas as pd
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px

# Load the dataset
spacex_df = pd.read_csv("spacex_launch_dash.csv")
min_payload = spacex_df['Payload Mass (kg)'].min()
max_payload = spacex_df['Payload Mass (kg)'].max()

# Initialize the Dash app
app = dash.Dash(__name__)

app.layout = html.Div(children=[
    html.H1('SpaceX Launch Records Dashboard', 
            style={'textAlign': 'center', 'color': '#503D36', 
                   'font-size': 40}),
    dcc.Dropdown(
        id='site-dropdown',
        options=[
            {'label': 'All Sites', 'value': 'ALL'},
            {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
            {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
            {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
            {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}
        ],
        value='ALL',
        placeholder="Select a Launch Site here",
        searchable=True
    ),
    html.Br(),
    html.Div(dcc.Graph(id='success-pie-chart')),
    html.Br(),
    html.P("Payload range (Kg):"),
    dcc.RangeSlider(
        id='payload-slider',
        min=0, max=10000, step=1000,
        marks={i: '{}'.format(i) for i in range(0, 10001, 1000)},
        value=[min_payload, max_payload]
    ),
    html.Div(dcc.Graph(id='success-payload-scatter-chart')),
])

@app.callback(
    [Output(component_id='success-pie-chart', component_property='figure'),
     Output(component_id='success-payload-scatter-chart', component_property='figure')],
    [Input(component_id='site-dropdown', component_property='value'),
     Input(component_id='payload-slider', component_property='value')]
)
def update_charts(selected_site, payload_range):
    # Filter data based on payload range
    low, high = payload_range
    filtered_df = spacex_df[(spacex_df['Payload Mass (kg)'] >= low) & 
                            (spacex_df['Payload Mass (kg)'] <= high)]
    
    # Update pie chart
    if selected_site == 'ALL':
        pie_fig = px.pie(spacex_df, values='class', 
                         names='Launch Site', 
                         title='Total Success Launches by Site')
    else:
        site_df = spacex_df[spacex_df['Launch Site'] == selected_site]
        pie_fig = px.pie(site_df, names='class', 
                         title=f'Total Success Launches for site {selected_site}')
    
    # Update scatter plot
    if selected_site == 'ALL':
        scatter_fig = px.scatter(filtered_df, x='Payload Mass (kg)', y='class',
                                 color='Booster Version Category',
                                 title='Correlation between Payload and Success for all Sites')
    else:
        site_df = filtered_df[filtered_df['Launch Site'] == selected_site]
        scatter_fig = px.scatter(site_df, x='Payload Mass (kg)', y='class',
                                 color='Booster Version Category',
                                 title=f'Correlation between Payload and Success for site {selected_site}')
    
    return pie_fig, scatter_fig

if __name__ == '__main__':
    app.run_server(debug=True)
